In [1]:
import os
import csv
import torch
import argparse
import numpy as np
from mlp import mlp
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from transformers import InputExample, InputFeatures
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer, BertModel
from transformers import glue_convert_examples_to_features as convert_examples_to_features

/Users/YZY/opt/anaconda3/envs/pytorch_cpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
config=BertConfig.from_pretrained('./model')
tokenizer=BertTokenizer.from_pretrained('./model')
model=BertModel.from_pretrained('./model',config=config)

Some weights of the model checkpoint at ./model were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def create_examples(lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    del lines[0]
    for (i, line) in enumerate(lines):
        guid = "%s-%s" % (set_type, i)
        # label = int(line[1])
        # in available.csv ,text is put at col:1,and don't need to be replaced with YZYHUST
        # text_a = line[2].replace("YZYHUST", ',')
        text_a=line[1]
        examples.append(
            InputExample(guid=guid, text_a=text_a, text_b=None, label=None))
    return examples

def Load_data(tokenizer,file_path):
    csv.field_size_limit(500 * 1024 * 1024)
    with open(file_path, 'r') as f:
        examples = create_examples(list(csv.reader(f)), 'predict')
    label_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    features = convert_examples_to_features(
        examples,
        tokenizer,
        label_list=label_list,
        max_length=256,
        output_mode="classification",
    )
    all_input_ids = torch.tensor([f.input_ids for f in features],
                                 dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features],
                                      dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features],
                                      dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_attention_mask,
                            all_token_type_ids)
    return DataLoader(dataset, batch_size=16)

In [6]:
file_path='./url/ip/ip_test.csv'
pred_dataloader = Load_data(tokenizer,file_path=file_path)
file=pd.read_csv(file_path)
label=file['label']
label.to_csv('./url/ip/test_label.csv',index=None)

/Users/YZY/opt/anaconda3/envs/pytorch_cpu/lib/python3.10/site-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [7]:
feature_list=[]
for batch in tqdm(pred_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids':batch[2]
            }
            seq_outputs,pool_outputs = model(**inputs,return_dict=False)
            feature_list.append(pool_outputs)
features=torch.concat(feature_list,dim=0)
torch.save(features,'./url/ip/features_test.pt')

Evaluating: 100%|██████████| 55/55 [03:40<00:00,  4.01s/it]


# 拼接特征，进行最后的预测